In [1]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://sjet-datamart-bucket/Jars/mssql-jdbc-6.1.0.jre8.jar",
        "spark.sql.crossJoin.enabled": "true"
    }
}

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1674104491510_0004,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1674104491510_0003,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1674104491510_0003,pyspark,idle,Link,Link,✔


In [3]:
Q400_512WPS_1 = spark.sql('''
                        SELECT
                        
                        ECN,
                        
                        UTC_TIMEDATE,
                        
                        FileName,

                        CAST((MAX(`Time-In-Secs`) - MIN(`Time-In-Secs`)) / 60 AS INT) AS FlightTime
             
                        FROM precogdb.q400512wps
                        
                        WHERE FlightDate > '2022-10-31'
                        
                        AND `Rad-Alt` NOT LIKE '%*%'
                        
                        AND `Rad-Alt` <> ''
                        
                        AND `WOW-MLG` = 'Air (0)'

                        --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)

                        GROUP BY ECN, UTC_TIMEDATE, Filename
                            ''')

Q400_512WPS_1.createOrReplaceTempView('Q400_512WPS_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Q400_512WPS_1.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
Q400_Time = spark.sql('''

                                SELECT CAST((MAX(CAST(`Time-in-secs` AS INT)) - MIN(CAST(`Time-in-secs` AS INT))) / 60 AS INT) AS BlockTime_DFDR, filename
                                
                                FROM precogdb.q400512wps
                                
                                WHERE FlightDate > '2022-10-31'

                                --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)
                                
                                group by filename
                                
                            ''')

Q400_Time.createOrReplaceTempView('Q400_Time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
Q400_512WPS_2 = spark.sql('''

                        SELECT 
                        
                            DISTINCT
                        
                               Cast(ChocksOff as Date) AS FlightDate,
                               
                               TailNo,
                               
                               Departure,
                               
                               ChocksOff,
                               
                               date_format(ChocksOff, 'HH:mm:ss') AS Actual_DEP_Time,
                               
                               Arrival,
                               
                               date_format(ChocksOn, 'HH:mm:ss') AS Actual_ARR_Time,
                               
                               (bigint(to_timestamp(ChocksOn)) - bigint(to_timestamp(ChocksOff))) / 60 AS BlockTime_ARMS, 
                               
                               CONCAT(Departure, '-', Arrival) AS Sector,
                                                    
                               desigflightnumber AS FlightNo,
                               
                               Date,
                               
                               ECN,
                               
                               UTC_TIMEDATE,
                               
                               CAST(SUM((CAST (`FUEL-FLOW-2-kg-h` AS INT) + CAST (`FUEL-FLOW-1-kg-h` AS INT))) OVER (PARTITION BY ECN, UTC_TIMEDATE, Filename) / 3600 AS INT) AS FuelConsumed,
                               
                               FileName,
                               
                               loadeddate,
                               
                               year,
                               
                               month
                               
                              FROM precogdb.q400512wps
                        
                        WHERE FlightDate > '2022-10-31'
                        
                        --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)
                        
                        AND `Rad-Alt` NOT LIKE '%*%'
                        
                        AND `Rad-Alt` <> ''
                        
                        AND UTC_TIMEDATE BETWEEN (ChocksOff - INTERVAL 360 MINUTES) AND (ChocksOff - INTERVAL 300 MINUTES)
             
                        ''')

Q400_512WPS_2.createOrReplaceTempView('Q400_512WPS_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#Q400_512WPS_2.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
Q400_512WPS_2_1 = spark.sql('''

                        SELECT a.*,b.Blocktime_DFDR
                        
                        FROM Q400_512WPS_2 a 
                        
                        left JOIN Q400_Time b
                        
                        ON a.filename = b.filename
                    ''')

Q400_512WPS_2_1.createOrReplaceTempView('Q400_512WPS_2_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Q400_512WPS_2_1.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
Q400_512WPS_3 = spark.sql('''

                SELECT MIN(`Time-Per-Sec`) AS AirborneTime,
                
                ECN,
                        
                UTC_TIMEDATE,
                
                flightdate,
                
                Filename
                
                FROM precogdb.q400512wps
                
                WHERE FlightDate > '2022-10-31'
                        
                AND `Rad-Alt` NOT LIKE '%*%'
                        
                AND `Rad-Alt` <> ''
                        
                AND `WOW-MLG` = 'Air (0)'

                --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)

                GROUP BY ECN,
                        
                        UTC_TIMEDATE,flightdate,Filename
''')

Q400_512WPS_3.createOrReplaceTempView('Q400_512WPS_3')

Q400_512WPS_3_1 = spark.sql('''SELECT MIN(`Time-Per-Sec`) AS TouchDownTime,
                
                flightdate,
                
                ECN,
                        
                UTC_TIMEDATE,
                
                Filename
                
                FROM precogdb.q400512wps
                
                WHERE FlightDate > '2022-10-31'
                        
                AND `Rad-Alt` NOT LIKE '%*%'
                        
                AND `Rad-Alt` <> ''
                
                AND `WOW-MLG` = 'GROUND (1)'
                
                AND `time-per-sec` NOT LIKE '%*%'
                
                AND `time-in-secs` > 2500
                        
                --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)

                GROUP BY ECN,
                        
                        UTC_TIMEDATE,flightdate,Filename ''')

Q400_512WPS_3_1.createOrReplaceTempView('Q400_512WPS_3_1')

Q400_512WPS_3_2 = spark.sql('''

            SELECT a.ECN, a.UTC_TIMEDATE, a.AirborneTime, b.TouchDownTime, a.Filename, a.Flightdate
            FROM Q400_512WPS_3 a 
            JOIN Q400_512WPS_3_1 b on a.Filename = b.Filename
            AND a.Flightdate = b.Flightdate
''')

Q400_512WPS_3_2.createOrReplaceTempView('Q400_512WPS_3_2')

Q400_512WPS_4 = spark.sql('''

                            SELECT ECN, UTC_TIMEDATE, filename, flightdate,
                            
                            CASE 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 1 THEN CONCAT('00000',CAST(`AirborneTime` as String)) 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 2 THEN CONCAT('0000',CAST(`AirborneTime` as String)) 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 3 THEN CONCAT('000',CAST(`AirborneTime` as String))
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 4 THEN CONCAT('00',CAST(`AirborneTime` as String)) 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 5 THEN CONCAT('0',CAST(`AirborneTime` as String))
                                ELSE `AirborneTime`
                            END As AirborneTime,
                            CASE 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 1 THEN CONCAT('00000',CAST(`TouchdownTime` as String)) 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 2 THEN CONCAT('0000',CAST(`TouchdownTime` as String)) 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 3 THEN CONCAT('000',CAST(`TouchdownTime` as String))
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 4 THEN CONCAT('00',CAST(`TouchdownTime` as String)) 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 5 THEN CONCAT('0',CAST(`TouchdownTime` as String))
                                ELSE `TouchdownTime`
                            END As TouchdownTime
                            
                                                                
                            FROM Q400_512WPS_3_2
                                                
                    ''')

Q400_512WPS_4.createOrReplaceTempView('Q400_512WPS_4')

Q400_512WPS_5 = Q400_512WPS_4.withColumn("AirborneTime", regexp_replace(col("AirborneTime") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" )) \
                            .withColumn("TouchdownTime", regexp_replace(col("TouchdownTime") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" ))

Q400_512WPS_5.createOrReplaceTempView('Q400_512WPS_5')

Q400_512WPS_6 = spark.sql('''

                SELECT to_timestamp(concat(flightdate,' ',AirborneTime)) AS Airborne_Time,
                
                to_timestamp(concat(flightdate,' ',TouchdownTime)) AS Touchdown_Time,
                
                Filename,
                
                ECN, 
                
                UTC_TIMEDATE
                
                FROM Q400_512WPS_5


''')

Q400_512WPS_6.createOrReplaceTempView('Q400_512WPS_6')

Q400_512WPS_3_1.show(truncate = False)

Q400_512WPS_7 = spark.sql('''

                SELECT (Airborne_Time + INTERVAL 330 MINUTES) AS AirborneTime, 
                
                (Touchdown_Time + INTERVAL 330 MINUTES) AS TouchdownTime,
                
                Filename,
                
                ECN, UTC_TIMEDATE
                
                FROM Q400_512WPS_6
                
                ''')

Q400_512WPS_8 = Q400_512WPS_7.withColumn('Airborne_Time', date_format('AirborneTime', 'HH:mm:ss')) \
                            .withColumn('Touchdown_Time', date_format('TouchdownTime', 'HH:mm:ss'))

Q400_512WPS_8.createOrReplaceTempView('Q400_512WPS_8')#show(truncate = False)

Q400_FOB_1 = spark.sql('''

                    SELECT 
                    
                    b.flightdate,
                               
                               b.TailNo,
                               
                               b.Departure,
                               
                               b.Actual_DEP_Time,
                               
                               c.Airborne_Time,
                               
                               c.Touchdown_Time,
                               
                               b.Arrival,
                               
                               b.Sector,
                               
                               b.Actual_ARR_Time,
                               
                               b.Blocktime_ARMS,
                               
                               b.Blocktime_DFDR,
                               
                               a.FlightTime,
                                                    
                               b.FlightNo,
                               
                               b.Date,
                               
                               b.FuelConsumed,
                               
                               b.FileName,
                               
                               b.loadeddate,
                               
                               b.year,
                               
                               b.month
                               
                               FROM Q400_512WPS_1 a
                               
                               INNER JOIN Q400_512WPS_2_1 b
                               
                               INNER JOIN Q400_512WPS_8 c
                               
                               on a.filename = b.filename
                               
                               and a.ECN = b.ECN
                               
                               and a.UTC_TIMEDATE = b.UTC_TIMEDATE
                               
                               and a.filename = c.filename
                               
                               and a.ECN = c.ECN
                               
                               and a.UTC_TIMEDATE = c.UTC_TIMEDATE
                               
                               --where a.filename = 'ALL ac=VT-SUQ dep=PNY arr=HYD fn=SG 3997 dt=20221122.csv'
                               
                               
                               
                               ''')

Q400_FOB_1.createOrReplaceTempView('Q400_FOB_1')

Q400_FOB_1.show(truncate=False)

Q400_FOB = spark.sql('''

    SELECT 
                    
                    flightdate,
                               
                               TailNo,
                               
                               Departure,
                               
                               Actual_DEP_Time,
                               
                               Airborne_Time,
                               
                               Touchdown_Time,
                               
                               Arrival,
                               
                               Sector,
                               
                               Actual_ARR_Time,
                               
                               CONCAT( CASE 
                                    WHEN LENGTH ( CAST(CAST(Blocktime_ARMS / 60 AS INT) AS STRING)) =1
                                    THEN CONCAT('0',CAST(CAST(Blocktime_ARMS / 60 AS INT) AS STRING))
                                    ELSE CAST(CAST(Blocktime_ARMS / 60 AS INT) AS STRING)
                                END,
                                ':',
                                CASE 
                                    WHEN LENGTH ( CAST(CAST(Blocktime_ARMS % 60 AS INT) AS STRING)) =1
                                    THEN CONCAT('0',CAST(CAST(Blocktime_ARMS % 60 AS INT) AS STRING))
                                    ELSE CAST(CAST(Blocktime_ARMS % 60 AS INT) AS STRING)
                                END ) AS BlockTime_ARMS,
                               
                                CONCAT( CASE 
                                    WHEN LENGTH ( CAST(CAST(Blocktime_DFDR / 60 AS INT) AS STRING)) =1
                                    THEN CONCAT('0',CAST(CAST(Blocktime_DFDR / 60 AS INT) AS STRING))
                                    ELSE CAST(CAST(Blocktime_DFDR / 60 AS INT) AS STRING)
                                END,
                                ':',
                                CASE 
                                    WHEN LENGTH ( CAST(CAST(Blocktime_DFDR % 60 AS INT) AS STRING)) =1
                                    THEN CONCAT('0',CAST(CAST(Blocktime_DFDR % 60 AS INT) AS STRING))
                                    ELSE CAST(CAST(Blocktime_DFDR % 60 AS INT) AS STRING)
                                END ) AS BlockTime_DFDR,
                               
                                CONCAT( CASE 
                                    WHEN LENGTH ( CAST(CAST(FlightTime / 60 AS INT) AS STRING)) =1
                                    THEN CONCAT('0',CAST(CAST(FlightTime / 60 AS INT) AS STRING))
                                    ELSE CAST(CAST(FlightTime / 60 AS INT) AS STRING)
                                END,
                                ':',
                                CASE 
                                    WHEN LENGTH ( CAST(CAST(FlightTime % 60 AS INT) AS STRING)) =1
                                    THEN CONCAT('0',CAST(CAST(FlightTime % 60 AS INT) AS STRING))
                                    ELSE CAST(CAST(FlightTime % 60 AS INT) AS STRING)
                                END ) AS FlightTime,
                                                    
                                FlightNo,
                               
                                Date,
                               
                                FuelConsumed,
                               
                                FileName,
                               
                                loadeddate,
                               
                                year,
                               
                                month

                                FROM Q400_FOB_1
    

    
    
    ''')

Q400_FOB.createOrReplaceTempView('Q400_FOB')

SALTDF = spark.read \
  .format("jdbc") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=SALT") \
  .option("dbtable", "LoadAndTrimDetails") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .load()

SALTDF.createOrReplaceTempView('SALTDF')

Q400FinalFOBDF = spark.sql('''

                SELECT a.flightdate,
                               
                               a.TailNo,
                               
                               a.FlightNo,
                               
                               a.Sector,
                               
                               a.Departure,
                               
                               a.Actual_DEP_Time,
                               
                               a.Airborne_Time,
                               
                               a.Touchdown_Time,
                               
                               a.Arrival,
                               
                               a.Actual_ARR_Time,
                               
                               b.FOB,
                               
                               (CAST (b.FOB AS INT) + 105) AS DepartureFuel,
                               
                               a.FuelConsumed,
                               
                               a.BlockTime_ARMS,
                               
                               a.BlockTime_DFDR,
                               
                               a.FlightTime,
                                                    
                               ((CAST (b.FOB AS INT) + 105) - CAST (a.FuelConsumed AS INT)) AS ArrivalFuel,
                               
                               a.FileName,
                               
                               a.loadeddate,
                               
                               a.year,
                               
                               a.month,

                               current_timestamp()   AS `ProcessedDate`

                               
                               
                               
                
                FROM Q400_FOB a 
                
                LEFT JOIN SALTDF b on a.FlightDate = CAST(b.FlightDate AS date)
                
                AND REPLACE(a.FlightNo,' ','') = b.FlightNo
                
                AND a.Departure = b.Source
                
                AND a.Arrival = b.Destination
                
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+------+-------------------+--------------------------------------------------------+
|TouchDownTime|flightdate|ECN   |UTC_TIMEDATE       |Filename                                                |
+-------------+----------+------+-------------------+--------------------------------------------------------+
|92852        |2022-11-02|105317|2022-11-02 07:39:38|ALL ac=VT-SQJ dep=BLR arr=SAG fn=SG 4005 dt=20221102.csv|
|130119       |2023-01-03|105317|2023-01-03 11:25:40|ALL ac=VT-SUR dep=SAG arr=BLR fn=SG 4004 dt=20230103.csv|
|73529        |2022-12-09|106102|2022-12-09 06:47:32|ALL ac=VT-SUR dep=HJR arr=VNS fn=SG 2931 dt=20221209.csv|
|94727        |2022-11-01|106235|2022-11-01 08:30:20|ALL ac=VT-SQG dep=DEL arr=JDH fn=SG 2905 dt=20221101.csv|
|92420        |2023-01-12|106869|2023-01-12 08:19:08|ALL ac=VT-SQG dep=DHM arr=DEL fn=SG 3236 dt=20230112.csv|
|55443        |2023-01-13|107167|2023-01-13 04:34:24|ALL ac=VT-SUR dep=HYD arr=SAG fn=SG 4003 dt=20230113.csv|
|

In [11]:
Q400FinalFOBDF.createOrReplaceTempView('Q400FinalFOBDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#Q400FinalFOBDF.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
FuelOnBoardFinal = spark.sql(''' SELECT 
                                       FlightDate
                                      ,TailNo
                                      ,FlightNo
                                      ,Sector
                                      ,Departure
                                      ,Actual_DEP_time
                                      ,Airborne_Time
                                      ,Touchdown_Time
                                      ,Arrival
                                      ,Actual_ARR_time
                                      ,fob
                                      ,DepartureFuel
                                      ,FuelConsumed
                                      ,Blocktime_ARMS
                                      ,Blocktime_DFDR
                                      ,FlightTime
                                      ,ArrivalFuel
                                      ,FileName
                                      ,LoadedDate
                                      ,current_timestamp() AS `ProcessedDate`
                                      ,Year
                                      ,Month
                                FROM Q400FinalFOBDF ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
FuelOnBoardFinal.write.mode('append').partitionBy("year", "month").parquet('s3://sjet-datamart-bucket/FuelOnBoard/fuel-on-board-processed-q400/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
FuelOnBoardFinal \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=FuelDashBoard") \
  .option("dbtable", "FuelOnBoard_Q400STG") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
FuelOnBoardFinal.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2783

In [17]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
df1 = spark.read.parquet('s3://sjet-datamart-bucket/FuelOnBoard/fuel-on-board-processed-q400/year=2023/month=January/part-00000-700249a6-092a-4129-b110-06215f322b1f.c000.snappy.parquet')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1674017527261_0029,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df1.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+--------+-------+---------+---------------+-------------+--------------+-------+---------------+--------+-------------+------------+--------------+--------------+----------+-----------+---------------------------------------------------------+-----------------------+-----------------------+
|flightdate|tailno|flightno|sector |departure|actual_dep_time|airborne_time|touchdown_time|arrival|actual_arr_time|salt_fob|departurefuel|fuelconsumed|blocktime_arms|blocktime_dfdr|flighttime|arrivalfuel|filename                                                 |loadeddate             |processeddate          |
+----------+------+--------+-------+---------+---------------+-------------+--------------+-------+---------------+--------+-------------+------------+--------------+--------------+----------+-----------+---------------------------------------------------------+-----------------------+-----------------------+
|2023-01-13|VT-SUR|SG 1006 |HYD-ISK|HYD      |06:10:00       |06:32

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…